---
jupyter:
  jupytext:
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
---

<!-- #region id="0487f37c" -->
# Table des matières
1. [Rappel: différences entre la régression et la classification](#rappel-différences-entre-la-régression-et-la-classification)
1. [Modèle de la régression logistique](#modèle-de-la-régression-logistique)
1. [Différences et similarités entre la régression logistique et la régression linéaire](#différences-et-similarités-entre-la-régression-logistique-et-la-régression-linéaire)
1. [Applications](#applications)
1. [Origine du modèle de régression logistique](#origine-du-modèle-de-régression-logistique)
1. [Estimation des paramètres de la régression logistique](#estimation-des-paramètres-de-la-régression-logistique)
  1. [Quelle est l'intuition derrière la fonction de perte?](#quelle-est-lintuition-derrière-la-fonction-de-perte)
    1. [Que vaut la perte $S_i$ lors d'une classification parfaite?](#que-vaut-la-perte-s_i-lors-dune-classification-parfaite)
    1. [Que vaut la perte $S_i$ lors d'une mauvaise classification?](#que-vaut-la-perte-s_i-lors-dune-mauvaise-classification)
    1. [En résumé](#en-résumé)
1. [Exemple de classification 2-D](#exemple-de-classification-2-d)
1. [Identification des facteurs importants](#identification-des-facteurs-importants)
<!-- #endregion -->



In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import LogisticRegression

sns.set(color_codes=True)


seed = 42
np.random.seed(seed)


<!-- #region id="e93269f3" -->
Voilà une belle source de confusion; la régression logistique est en fait une méthode de ... classification!
L'origine historique de cette confusion de nom est expliquée plus loin. On peut dire que c'est une méthode d'analyse
des données à la frontière entre la régression et la classification.
<!-- #endregion -->

<!-- #region id="ad590c8f" -->
# <a id=rappel-différences-entre-la-régression-et-la-classification>Rappel: différences entre la régression et la classification</a>
<!-- #endregion -->

<!-- #region id="34aa2e7d" -->
La régression sert à prédire une réponse $y$, qui est une variable numérique continue, en fonction de plusieurs
facteurs $x_{i}$

$$y=f(x_{1}, \cdots, x_{N}, \Theta)$$

où $\Theta$ représente l'ensemble des paramètres de la fonction $f$.

La classification, quant à elle, sert à prédire une classe $C$, qui est une variable catégorielle, en fonction de
plusieurs facteurs $x_{i}$

$$C=g(x_{1}, \cdots, x_{N}, \Theta)$$

où $\Theta$ représente l'ensemble des paramètres de la fonction $g$.

La régression logistique chevauche les deux; elle prédit la probabilité de la classe $C$.
<!-- #endregion -->

<!-- #region id="61fbb3b4" -->
# <a id=modèle-de-la-régression-logistique>Modèle de la régression logistique</a>
<!-- #endregion -->

<!-- #region id="1b0a77ea" -->
Ainsi, la régression logistique sert à prédire la probabilité $p \in [0,1]$ de succès (1, "Vrai", "True") qu'un
évènement $E$ se produise, en fonction de plusieurs facteurs $x_{i}$. La formulation standard du modèle est

$$p(E|X,\Theta)= {\dfrac {1}{1+e^{-(a_{0}+a_{1}x_{1}+\cdots + a_{N} x_{N})}}}$$

où les $a_{i}$ sont les paramètres de la fonction.

Dans le cas plus général, on peut remplacer l'expression précédente avec

$$p(E\vert X,\Theta) = {\dfrac {1}{1+e^{-f(x_{1},\cdots, x_{N},\Theta)}}}$$

où $\Theta$ représente l'ensemble des paramètres de la fonction $f$. Celle-ci est généralement
un polynôme de degré 1 ou 2.
<!-- #endregion -->

<!-- #region id="01e8d982" -->
Lorsque $p(E)>\tau$, où $\tau$ est un seuil de probabilité prédéterminé, on considère que l'évènement E s'est
produit et l'étiquette de la classe résultante est (1, "Vrai", "True", A, etc.). Dans le cas contraire, l'étiquette est
(0, "Faux", "False", B, etc.) Il est à noter que le seuil de probabilité $\tau \in [0,1]$; il peut être différent de $50~\%$.
<!-- #endregion -->

<!-- #region id="ba75bd6f" -->
# <a id=différences-et-similarités-entre-la-régression-logistique-et-la-régression-linéaire>Différences et similarités entre la régression logistique et la régression linéaire</a>
<!-- #endregion -->

<!-- #region id="afe8ce74" -->
La figure suivante compare les deux méthodes. Dans le panneau de
droite, on voit un exemple classique de régression linéaire où les paramètres d'une droite ont été ajustés
afin de la recaler sur les données (*fit a linear model to the data*). Le panneau
de gauche montre une application totalement différente. Dans ce cas-ci, la droite sert de frontière entre
les deux classes de points. C'est donc un problème de classification. C'est le genre de frontière que l'on
observe lorsque l'on utilise la régression logistique avec une fonction $f(x_{1},\cdots, x_{N},\Theta)$ linéaire.

<!-- #endregion -->

<!-- #region id="9a5f3e2d" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/logistic-vs-linear-regression.png"  width="500" />
    <div>
    <font size="0.5">Image Source: https://www.researchgate.net/publication/335786324_Regression_Analysis_With_Differential_Privacy_Preserving/figures?lo=1/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="694c7fdd" -->
# <a id=applications>Applications</a>
<!-- #endregion -->

<!-- #region id="38f3d072" -->
La régression logistique est très utilisée en sciences sociales, en médecine, en finances.
On s'en sert afin de prédire la probabilité $p(E)$ qu'un évènement $E$ se produise alors que
celui-ci résulte le l'effet combiné d'une multitude de facteurs $x_{i}$. Ces facteurs peuvent être des
variables continues, catégorielles ou ordinales comme pour les modèles de régression
multilinéaires présentés dans les sections précédentes.

Voici quelques situations dans lesquelles la régression logistique est utilisée. Chacune dépend de multiples facteurs
sociétaux, financiers, technologiques, physiologiques, psychologiques, etc. Il faut pouvoir les mesurer
afin de pouvoir ensuite modéliser la probabilité de l'évènement étudié.
<!-- #endregion -->

<!-- #region id="b89098cc" -->
## Hôtellerie:
<!-- #endregion -->

<!-- #region id="b398c6bc" -->
- Quelle est la probabilité qu'un touriste canadien loue une chambre dans un hôtel en Égypte à
partir de ses réponses à un questionnaire en ligne?
<!-- #endregion -->

<!-- #region id="864f2351" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/cairo-egypt.jpeg"  width="300" />
    <div>
    <font size="0.5">Image Source: https://www.publicdomainpictures.net/en/view-image.php?image=354011&picture=cairo-travel-poster/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="ee44f8e2" -->
## Jeux vidéo:
<!-- #endregion -->

<!-- #region id="8f883892" -->
- Quelle est la probabilité qu'un joueur achète différents types de pouvoirs?
<!-- #endregion -->

<!-- #region id="394cc587" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/video-game-phone.jpeg"  width="300" />
    <div>
    <font size="0.5">Image Source: https://www.pexels.com/photo/crop-faceless-man-playing-video-game-on-smartphone-4293307/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="ff9b39d0" -->
## Sécurité:
<!-- #endregion -->

<!-- #region id="eec99309" -->
- Quelle est la probabilité d'échec d'un lancement de fusée?
- Quelle est la probabilité que le taux de criminalité dans une ville augmente?
<!-- #endregion -->

<!-- #region id="396cebb8" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/burglar.jpeg"  width="300" />
    <div>
    <font size="0.5">Image Source: https://pxhere.com/en/photo/1589039</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="79ecb9f5" -->
## Médecine:
<!-- #endregion -->

<!-- #region id="73ad3990" -->
- Au triage à l'urgence, il faut estimer la probabilité qu'une personne blessée survive à une opération.
<!-- #endregion -->

<!-- #region id="f4fa7f68" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/medecine-personel.png"  width="300" />
    <div>
    <font size="0.5">Image Source: https://www.peoplematters.in/article/talent-management/heres-a-look-at-people-management-issues-in-healthcare-18553</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="a66bb1d4" -->
## Sciences sociales:
<!-- #endregion -->

<!-- #region id="72b136f9" -->
- Quelle est la probabilité de retourner aux études et de compléter son programme?
- Quelle était la probabilité de survie d'un passager lors du naufrage du Titanic?
<!-- #endregion -->

<!-- #region id="0273c0ca" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/titanic-ship.jpeg"  width="300" />
    <div>
    <font size="0.5">Image Source: https://publicdomainpictures.net/en/view-image.php?image=63492&picture=wallpaper-clip-art</font>
    </div>
</div>
<p>&nbsp;</p>
<!-- #endregion -->

<!-- #region id="4c19f122" -->
# <a id=origine-du-modèle-de-régression-logistique>Origine du modèle de régression logistique</a>
<!-- #endregion -->

<!-- #region id="3f33f9b9" -->
On sait tous par expérience qu'on améliore nos chances de passer un examen si on étudie à l'avance. Elles
sont faibles si on étudie peu ou pas, et s'améliorent avec le nombre d'heures consacrées à l'étude. On sait également
que le succès n'est pas assuré même avec un nombre d'heures important. Comment peut-on modéliser ce phénomène basé sur
les observations de générations d'étudiants avant nous?

Faisons l'hypothèse que la seule variable réellement importante $x$ est le nombre d'heures étudiées et que la réponse à prédire $y$ est la probabilité de réussite. La figure suivante
montre les résultats d'un examen donné avec l'évènement $E=\{0, 1\}$ où $E=1$ implique un examen réussi. Les deux observations
précédentes sont reproduites; plus on étudie, meilleures sont les chances de succès, mais cela n'empêche pas
certains étudiants d'échouer malgré plusieurs heures consacrées à l'étude. De plus, pour un même nombre d'heures étudiées,
certains passent, d'autres non.
<!-- #endregion -->

<!-- #region id="59fedc69" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/logistic-regression-origin.jpeg"  width="500" />
    <div>
    <font size="0.5">Image Source: https://en.wikipedia.org/wiki/Logistic_regression#Applications</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="fc7f4d87" -->
La distribution inhabituelle des points $(x,y)$ indique qu'on ne peut pas modéliser $y$ comme une simple relation
univoque $y=f(x)$, car plusieurs valeurs de $y$ correspondent à une même valeur de $x$. La courbe représente
la probabilité de passage de l'examen; c'est ce que nous devons modéliser. Elle a une forme en "S"
et porte le nom de sigmoïde pour cette raison.

> À noter que la lettre grecque sigma correspond à la lettre latine $s$.

La fonction logistique est un exemple de fonction sigmoïde couramment utilisée. Elle est définie comme suit.

$$y = \frac{1}{1+e^{-x}} $$

La fonction logistique est bornée entre 0 et +1 lorsque x varie entre $-\infty$ et $+\infty$. On l'utilise
généralement en remplaçant l'argument $x$ par une fonction arbitraire $f(x,\Theta)$ qui augmente ou
diminue de façon monotone.

$$y = \frac{1}{1+e^{-f(x,\Theta)}} $$
où $\Theta$ représente l'ensemble des coefficients de la fonction $f$.

Cette fonction est utilisée à la fois dans les problèmes de régression et de classification. Dans les deux cas,
les coefficients $\Theta$ doivent être ajustés afin de recaler des données expérimentales à un modèle. La
similarité entre les deux approches contribue à la confusion associée au nom de la méthode de classification.

On retrouve souvent la fonction logistique en biologie. On s'en sert pour modéliser les phénomènes de croissance des populations bactérienne et virale en fonction du temps. C'est un exemple de régression où l'on cherche à recaler un modèle théorique de croissance à des données $(N,t)$ expérimentales où $N$ est le nombre d'organismes observés après un temps $t$.

<!-- #endregion -->

<!-- #region id="0ccb4f2f" -->
# <a id=estimation-des-paramètres-de-la-régression-logistique>Estimation des paramètres de la régression logistique</a>
<!-- #endregion -->

<!-- #region id="d5fd7fe6" -->
En régression logistique, la fonction à minimiser est l'entropie croisée (*cross-entropy loss*). Celle-ci provient d'un modèle linéaire généralisé avec une distribution de Bernouilli.

$$S(\Theta)= S(a_{0}, a_{1}, \cdots, a_{N})=-\frac{1}{n}\sum\limits_{i=1}^{n}[y_{i}\log{\hat y_{i}} +(1-y_{i})\log{(1-\hat y_{i})}]$$

où les $y_{i}=\{0, 1\}$ sont les observations et les $\hat y_{i}$ sont les prédictions du modèle général
utilisé en régression logistique multivariable

$$\hat y_{i} = p(1\vert X_{i},\Theta) = {\dfrac  {1}{1+e^{-f(x_{1},\cdots, x_{N},\Theta)}}}.$$

Contrairement à la méthode des moindres carrés, on ne peut obtenir d'expressions exactes pour les paramètres
$\Theta$ en minimisant l'entropie croisée avec $\nabla S(\Theta) = 0$. Il faut utiliser des méthodes d'optimisation numérique. La classe
[`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
de la librairie Scikit-Lean calcule les paramètres $\Theta$  pour nous.
<!-- #endregion -->

<!-- #region id="d144e94e" -->
## <a id=quelle-est-lintuition-derrière-la-fonction-de-perte>Quelle est l'intuition derrière la fonction de perte?</a>
<!-- #endregion -->

<!-- #region id="59beff2f" -->
Voyons comment elle fonctionne. Pour une donnée à classifier $(X_i,y_i)$ on a
$$S_i = -[y_{i}\log{\hat y_{i}} +(1-y_{i})\log{(1-\hat y_{i})}]$$

que l'on peut réécrire comme suit

$$S_i = -y_{i}\log{p(1|X)} -(1-y_{i})\log{p(0|X)}.$$


Si la classe réelle vaut 1 ($y_i=1$), on a
$$S_i = -\log{p(1|X)}.$$


Si la classe réelle vaut 0, ($y_i=0$), on a
$$S_i = -\log{p(0|X)}.$$

<!-- #endregion -->

<!-- #region id="3b8516b0" -->
### <a id=que-vaut-la-perte-s_i-lors-dune-classification-parfaite>Que vaut la perte $S_i$ lors d'une classification parfaite?</a>
<!-- #endregion -->

<!-- #region id="609506e1" -->
Dans ce cas on a $p(0|X)=1$ ou $p(1|X)=1$, soit

$$S_i=-\log{1}=0.$$

La perte associée est nulle, quelle que soit la classe.
<!-- #endregion -->

<!-- #region id="48772d62" -->
### <a id=que-vaut-la-perte-s_i-lors-dune-mauvaise-classification>Que vaut la perte $S_i$ lors d'une mauvaise classification?</a>
<!-- #endregion -->

<!-- #region id="53a6b502" -->
Faisons l'hypothèse que la classification est tellement mauvaise que la probabilité associée est de $1~\%$.Dans ce cas on a la prochaine équation, quelle que soit la classe

$$S_i=-\log{0,01}=2$$

où on a utilisé un logarithme en base 10 pour simplifier l'analyse. La perte est d'autant plus grande que la probabilité est faible.
<!-- #endregion -->

<!-- #region id="376bd30f" -->
### <a id=en-résumé>En résumé</a>
<!-- #endregion -->

<!-- #region id="9efd8c3e" -->
La fonction de perte logistique assigne une valeur nulle aux bonnes classifications et une valeur positive
aux mauvaises classifications. En minimisant la perte totale, $S=\sum\limits_{i=1}^{n}S_i$, on force le classificateur à faire de bonnes classifications. Il fallait y penser!

<!-- #endregion -->

<!-- #region id="47b0aebf" -->
# <a id=exemple-de-classification-2-d>Exemple de classification 2-D</a>
<!-- #endregion -->

<!-- #region id="d6b3bcc3" -->
Dans ce suit, nous allons montrer un exemple simple où la réponse $y$
dépend de deux variables $x_{1}$ et $x_{2}$. On a

$$p(1|X,\Theta) = {\dfrac  {1}{1+e^{-(a_{0}+a_{1}x_{1}+a_{2}x_{2})}}}$$

La frontière entre les classes 0 et 1 est située aux endroits où la probabilité $p(1|X,\Theta)=50\%$, ce qui se produit lorsque l'argument de l'exponentielle est nul. On obtient alors l'équation de la courbe délimitant la frontière entre les deux classes:

$$x_{2} = -\dfrac{(a_{0}+a_{1}x_{1})}{a_{2}}$$

Les coefficients $a_{1}$ sont calculés lors de l'entraînement du classificateur. Cet exemple s'inspire du
[blogue](https://thedatafrog.com/fr/articles/logistic-regression-neural-network/) suivant.

<!-- #endregion -->

<!-- #region id="48c852c3" -->
Pour l'exemple, on utilise deux distributions gaussiennes similaires se superposant légèrement. On assigne
les étiquettes 0 et 1 aux données composant chacune.
<!-- #endregion -->



In [ ]:
# Génération et affichage des données

# Distribution gaussienne en 2D
normal = np.random.multivariate_normal

# Génère 500 valeurs de x1 et de x2
n = 500
x1 = normal([-1.0, -1.0], [[1, 0.0], [0.0, 1]], n)
x2 = normal([1.0, 1.0], [[1, 0.0], [0.0, 1]], n)

# Génère les étiquettes de x1 et x2
y1 = np.zeros((n,))
y2 = np.ones((n,))

# Génère l'ensemble d'entraînement (X, y)
X_train = np.concatenate((x1, x2))
y_train = np.concatenate((y1, y2))



<!-- #region id="595e9c0f" -->
La figure suivante montre l'ensemble de données d'entraînement composé des deux nuages de points appartenant
aux classes 0 et 1.
<!-- #endregion -->



In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(111)
ax.scatter(x1[:, 0], x1[:, 1], alpha=0.5, label="classe 0")
ax.scatter(x2[:, 0], x2[:, 1], alpha=0.5, label="classe 1")
ax.set_xlabel("x1", fontsize=20)
ax.set_ylabel("x2", fontsize=20, rotation=0)
ax.legend(loc="upper left")


In [ ]:
# Génération d'une grille couvrant la même plage de valeurs en $x_{1}$ et $x_{2}$

xmin, xmax, npoints = (-5, 5, 51)
linx1 = np.linspace(xmin, xmax, npoints)

gridx1, gridx2 = np.meshgrid(linx1, linx1)
grid = np.c_[gridx1.flatten(), gridx2.flatten()]



<!-- #region id="a07e6a67" -->
Dans la prochaine cellue, nous initialisons le classificateur à l'aide de Scikit-learn. Nous utilisons la fonction d'optimisation numérique LBFGS pour estimer les paramètres du modèle.
<!-- #endregion -->



In [ ]:
clf = LogisticRegression(solver="lbfgs")


In [ ]:
# Entraînement du classificateur

clf.fit(X_train, y_train)



<!-- #region id="493606d9" -->
Par la suite, nous effectuons les prédictions à l'aide du classificateur. Nous obtenons la probabilité locale qu'un point $(x_{1}, x_{2})$ appartienne à la classe 1.
<!-- #endregion -->



In [ ]:
prob = clf.predict_proba(grid)
z2 = prob[:, 1].reshape(npoints, npoints)



<!-- #region id="0061f3f0" -->
La figure suivante montre la distribution de probabilité d'appartenance à la classe 1 ($P(1|X)$) après
l'entraînement du classificateur.
<!-- #endregion -->



In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection="3d")
ax.plot_wireframe(gridx1, gridx2, z2, alpha=0.25, color="k")

ax.scatter3D(x1[:, 0], x1[:, 1], y1)
ax.scatter3D(x2[:, 0], x2[:, 1], y2)
ax.set_xlabel("x2", fontsize=20)
ax.set_ylabel("x1", fontsize=20)
ax.zaxis.set_rotate_label(False)
ax.set_zlabel("$P(1|X)$", rotation=90, fontsize=18)

ax.invert_xaxis()
ax.invert_yaxis()
ax.view_init(20, 35)



<!-- #region id="2c5d630e" -->
Comme dans le cas de la méthode des moindres carrés, on cherche à réduire la distance verticale entre chaque mesure $y_{i}$
et la surface modélisée. La régression logistique procède légèrement différemment. Si $y_{i}=0$, la distance est prise sous la courbe.
Si $y_{i}=1$, la distance est prise au-dessus de la courbe. La minimisation
de la fonction $S(\Theta)$ rapproche la surface des observations. C'est ce que l'on observe ci-dessus.

<!-- #endregion -->

<!-- #region id="aa5797b6" -->
Par la suite, nous affichons la zone d'influence de chaque classe. C'est-à-dire, la localisation de la zone où la probabilitée est de $50~\%$.
<!-- #endregion -->



In [ ]:
linx2 = -(clf.intercept_ + clf.coef_[0][0] * linx1) / clf.coef_[0][1]



<!-- #region id="5f41f13c" -->
La figure suivante montre les résultats précédents en couleurs en y superposant la frontière,
la ligne blanche pointillée, entre les deux classes.
<!-- #endregion -->



In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
im = ax.pcolor(
    gridx1, gridx2, prob[:, 1].reshape(npoints, npoints), shading="auto", cmap="viridis"
)
cb = plt.colorbar(im)
ax.scatter(x1[:, 0], x1[:, 1], alpha=0.5)
ax.scatter(x2[:, 0], x2[:, 1], alpha=0.5)
ax.set_xlabel("x1", fontsize=20)
ax.set_ylabel("x2", fontsize=20, rotation=0)
ax.plot(linx1, linx2, "w", linestyle="dashed")
ax.set_ylim([-5, 5])



<!-- #region id="47476328" -->
Si l'on utilisait maintenant un ensemble de test, chaque couple de valeurs
$(x_{1},x_{2})$ serait classifié selon la zone où il est situé. Ainsi, la
régression logistique sépare le domaine des variables $x_{1}$ et $x_{2}$ en deux zones lorsqu'il y a deux classes.
La frontière est linéaire, car la fonction $f(x_{1},x_{2},\Theta)$ est linéaire. Si l'on ajoutait le terme
d'interaction $x_{1}x_{2}$ et des puissances $x_{i}^2$, la frontière deviendrait plus complexe.

<!-- #endregion -->

<!-- #region id="ca14d30b" -->
# <a id=identification-des-facteurs-importants>Identification des facteurs importants</a>
<!-- #endregion -->

<!-- #region id="2434035b" -->
La régression logistique permet non seulement de prédire la probabilité d'un évènement $E$, mais aussi d'identifier les
principaux facteurs dans le modèle qui contrôlent ce choix. C'est possible si les facteurs ont été préalablement normalisés lors de l'étape
de prétraitement. Ce sujet est abordé plus en détail dans le module sur la méthodologie. Dans ce qui suit, nous
allons supposer que cette étape essentielle a été effectuée.

Selon la formulation standard de la régression logistique, on a

$$p(E|X, \Theta)= {\dfrac {1}{1+e^{-(a_{0}+a_{1}x_{1}+\cdots + a_{N} x_{N})}}}$$

Plus grande sera la valeur absolue d'un paramètre $a_{i}$, plus important sera le facteur $x_{i}$ associé! Si l'on ordonne
les valeurs des $|a_{i}|$ en ordre décroissant, on obtiendra les facteurs $x_{i}$ en ordre décroissant d'importance.
C'est ainsi que l'on détermine les facteurs $x_{i}$ d'intérêt dans un problème donné. L'effet de
chaque facteur $x_{i}$ est déterminé ensuite par le signe du paramètre associé $a_{i}$.

<!-- #endregion -->
